In [32]:
import spacy
from torchtext import data
import torch
import torch.nn as nn

In [33]:
# declare model class
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        embedded = self.dropout(self.embedding(x))
        
        output, (hidden, cell) = self.rnn(embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
            
        return self.fc(hidden.squeeze(0))

In [52]:
PATH = "model.pt"
INPUT_DIM = 25002
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

# create model class
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

# load saved model and evaluate
model.load_state_dict(torch.load(PATH))
model.eval()

# load vocabulary
PATH1 = "vocab.pt"
comment = torch.load(PATH1)

## Predict Sentiment

In [53]:
NLP = spacy.load('en')
def predict_sentiment(sentence):
    tokenized = [tok.text for tok in NLP.tokenizer(sentence)]
    indexed = [comment.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [54]:
sent = "I hate you"
predict_sentiment(sent)

0.8530766367912292